# 1.BERTのクラス分類で動詞を予測する分類問題のNN

In [2]:
# 手始めに、BERTのクラス分類で動詞を予測することとする

import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

import pickle


In [3]:
pip install tensorboard -U

Note: you may need to restart the kernel to use updated packages.


In [4]:
# データの前処理(データローダに入力できる形に整形)
# 分類する動詞のリスト
verb_list = ['ESTIMATE', 'COMINGOUT', 'DIVINATION', 'GUARD', 'VOTE',
            'ATTACK', 'DIVINED', 'IDENTIFIED', 'GUARDED', 'VOTED',
            'ATTACKED', 'AGREE', 'DISAGREE', 'Skip', 'Over' ] # REVIEW: Skip, Overをuppercaseにする必要があるかどうか

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# 文章をトークンに変換するトークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# ファインチューニングで学習するためミニバッチ生成用データローダ
# TODO: データを読み込んで前処理をする
# 1. textをトークンに変換
# 2. トークンにラベルを追加
# 3. dataset_for_loaderに追加
max_length = 128
dataset_for_loader = []

# データを取得
data_set_plain = pickle.load(open('dataset.pkl', 'rb'))
for data in data_set_plain:
    encoding = tokenizer(
            data[1],
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
    encoding['labels'] = verb_list.index(data[0].split()[1])
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

# #以下実行検証用学習データ
# test_plain_dataset = [
# ['ESTIMATE','私はAgent[10]が騎士だと推測する'],
# ['COMINGOUT','Agent[01]はAgent[03]が狂人だとカミングアウトする'],
# ['Over','Over'],
# ['COMINGOUT','Agent[01]が占いだとカミングアウトする'],
# ['COMINGOUT','Agent[01]はAgent[01]が占い師だとカミングアウトする'],
# ['DIVINED','私は占った結果Agent[01]は白だった'],
# ['DIVINED','Agent[01]が占った結果Agent[02]は人狼だった'],
# ['GUARD','Agent[01]を護衛する'],
# ]
# for data in test_plain_dataset:
#     encoding = tokenizer(
#             data[1],
#             max_length=max_length, 
#             padding='max_length',
#             truncation=True
#         )
#     encoding['labels'] = verb_list.index(data[0])
#     encoding = { k: torch.tensor(v) for k, v in encoding.items() }
#     dataset_for_loader.append(encoding)

print(dataset_for_loader[0])


{'input_ids': tensor([    2, 16204,  1865,  4314,  7535,  4118,    81,     7,     9, 16204,
         1865,  4314,   213,  4118,    14,  1076,    75,    13,  5705,    34,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [5]:
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [6]:
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [7]:
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='jp_to_prompt_model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=len(verb_list), lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (27)

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
# 検証データで確認
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /robot-qnap-1/okubo/dl_remote/work/aiwolf/jp_to_prompt_model/epoch=9-step=270.ckpt
ベストモデルの検証データに対する損失:  tensor(0.0188, device='cuda:0')


In [10]:
# 学習推移の確認
%load_ext tensorboard
%tensorboard --logdir ./

In [11]:
# テストデータで確認
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /robot-qnap-1/okubo/dl_remote/work/aiwolf/jp_to_prompt_model/epoch=9-step=270.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Loaded model weights from checkpoint at /robot-qnap-1/okubo/dl_remote/work/aiwolf/jp_to_prompt_model/epoch=9-step=270.ckpt
/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │            1.0            │
└───────────────────────────┴───────────────────────────┘

Accuracy: 1.00


In [12]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

# Transformers対応のモデルを./model_transformesに保存
# model.bert_sc.save_pretrained('./model_transformers') 
model.bert_sc.save_pretrained('./model_verb_predicter') 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [13]:
#保存したモデルが取得できるか確認
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_verb_predicter'
    # './model_transformers'
)
bert_sc = bert_sc.cuda()

In [14]:
# 学習したモデルを検証
text_list = [
    "Agent[05]はAgent[05]がボディーガードだとカミングアウトする",
    "Agent[15]が襲われたAgent[10]を霊媒すると白だった",
    "Agent[09]はAgent[09]が占い師だとカミングアウトする",
    "Agent[05]はAgent[05]がボディーガードだと自白する",
    "Agent[05]はAgent[05]がボディーガードだと暴露する",
    "Agent[01]はAgent[01]が人狼だと思う",
    "Agent[01]を吊るすべき",
    "私はAgent[01]に投票しようと思います",
    "Agent[01]がAgent[02]を護衛する",
    "Agent[01]がAgent[02]を守った", # <-汎化性能が低い。学習データに”護衛した”しかないからだと思われる
    "私はAgent[01]の考えに賛成です" # <-学習データにAGREEがないので、うまく行かない
]

# データの符号化
encoding = tokenizer(
        text_list,
        max_length=max_length, 
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

encoding = { k: v.cuda() for k, v in encoding.items() }

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1).cpu().numpy() # スコアが最も高いラベル
labels_predicted = [verb_list[i] for i in labels_predicted] # ラベルをラベル名に変換
print(labels_predicted)


['COMINGOUT', 'IDENTIFIED', 'COMINGOUT', 'COMINGOUT', 'COMINGOUT', 'ESTIMATE', 'ESTIMATE', 'VOTE', 'GUARDED', 'DIVINED', 'ESTIMATE']


In [16]:
# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1).cpu().numpy() # スコアが最も高いラベル
labels_predicted = [verb_list[i] for i in labels_predicted] # ラベルをラベル名に変換
for index, text in enumerate(text_list):
    print(f"predicted:{labels_predicted[index]} \t {text}")


predicted:COMINGOUT 	 Agent[05]はAgent[05]がボディーガードだとカミングアウトする
predicted:IDENTIFIED 	 Agent[15]が襲われたAgent[10]を霊媒すると白だった
predicted:COMINGOUT 	 Agent[09]はAgent[09]が占い師だとカミングアウトする
predicted:COMINGOUT 	 Agent[05]はAgent[05]がボディーガードだと自白する
predicted:COMINGOUT 	 Agent[05]はAgent[05]がボディーガードだと暴露する
predicted:ESTIMATE 	 Agent[01]はAgent[01]が人狼だと思う
predicted:ESTIMATE 	 Agent[01]を吊るすべき
predicted:VOTE 	 私はAgent[01]に投票しようと思います
predicted:GUARDED 	 Agent[01]がAgent[02]を護衛する
predicted:DIVINED 	 Agent[01]がAgent[02]を守った
predicted:ESTIMATE 	 私はAgent[01]の考えに賛成です


# 2.BERTのクラス分類で部分promptを推定するNN

## やるべき手順まとめ
### データセットの前処理
1. コーパスデータのインポート
2. prompt用のラベルを生成
3. 日本語とラベルをセットにしたデータ（辞書）を学習用データとして出力
### 学習
1. Chapter7のモデルを使って学習
### 検証
1. testデータでaccuraryの計算
2. モデルから得られた出力からpromptを復元して表示

In [ ]:
#やるべき手順まとめ
#データセットの作成
#   1.コーパスデータのインポート
#   2.prompt用のラベルと日本語がセットとなったデータセットの生成
#   3.データセットを学習用にデータローダに変換
#学習
#   1. Chapter7のモデルを使って学習
#検証
#   1.testデータでaccuraryの計算
#   2.モデルから得られた出力からpromptを復元して表示

In [1]:
#必要なパッケージのインポート
import random
import glob
import json
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# データ前処理用
import pickle
from aiwolfpy import ProtocolParser
from aiwolfpy.protocol.contents import *


/home/takuya/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/takuya/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# jp2prompt用のNNモデル
class BertForSequenceClassificationMultiLabel(torch.nn.Module):
    
    def __init__(self, model_name, num_labels):
        super().__init__()
        # BertModelのロード
        self.bert = BertModel.from_pretrained(model_name) 
        # 線形変換を初期化しておく
        self.linear = torch.nn.Linear(
            self.bert.config.hidden_size, num_labels
        ) 

    def forward(
        self, 
        input_ids=None, 
        attention_mask=None, 
        token_type_ids=None, 
        labels=None
    ):
        # データを入力しBERTの最終層の出力を得る。
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        last_hidden_state = bert_output.last_hidden_state
        
        # [PAD]以外のトークンで隠れ状態の平均をとる
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)
        
        # 線形変換
        scores = self.linear(averaged_hidden_state) 
        
        # 出力の形式を整える。
        output = {'logits': scores}

        # labelsが入力に含まれていたら、損失を計算し出力する。
        if labels is not None: 
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output['loss'] = loss
            
        # 属性でアクセスできるようにする。
        output = type('bert_output', (object,), output) 

        return output

## データセットの前処理

In [11]:
# 分類するラベルのリスト
subject_list = ["Agent[01]","Agent[02]", "Agent[03]", "Agent[04]", "Agent[05]",
                   "Agent[06]", "Agent[07]", "Agent[08]", "Agent[09]", "Agent[10]", 
                   "Agent[11]", "Agent[12]", "Agent[13]", "Agent[14]", "Agent[15]","UNSPEC","ANY"] #TODO:ここ周りテキトーにやってる。ホントは分類ではなく値自身を使えばいいはず
verb_list = ['ESTIMATE', 'COMINGOUT', 'DIVINATION', 'GUARD', 'VOTE',
            'ATTACK', 'DIVINED', 'IDENTIFIED', 'GUARDED', 'VOTED',
            'ATTACKED', 'AGREE', 'DISAGREE', 'Skip', 'Over' ] # REVIEW: Skip, Overをuppercaseにする必要があるかどうか
target_list = subject_list
species_list = ['HUMAN',"WEREWOLF","ANY"]
role_list = ['VILLAGER','SEER', 'MEDIUM','BODYGUARD','WEREWOLF','POSSESSED','ANY']
talk_number_list = [str(i) for i in range(1, 16)] #TODO:ここ周りテキトーにやってる。ホントは分類ではなく値自身を使えばいいはず


# ラベルの数
label_size = len(subject_list) + len(verb_list) + len(target_list) \
                    + len(role_list) + len(species_list) + len(talk_number_list)
# ラベル用ディクショナリの生成
cum_sum = 0
subject_start_index = cum_sum
subject_dict = {subject_list[i]:i for i in range(len(subject_list))}
cum_sum += len(subject_list)
subject_end_index = cum_sum
verb_start_index = cum_sum

verb_dict = {verb_list[i]:i+cum_sum for i in range(len(verb_list))}
cum_sum += len(verb_list)
verb_end_index = cum_sum
target_start_index = cum_sum

target_dict = {target_list[i]:i+cum_sum for i in range(len(target_list))}
cum_sum += len(target_list)
target_end_index = cum_sum
role_start_index = cum_sum

role_dict = {role_list[i]:i+cum_sum for i in range(len(role_list))}
cum_sum += len(role_list)
role_end_index = cum_sum
species_start_index = cum_sum

species_dict = {species_list[i]:i+cum_sum for i in range(len(species_list))}
cum_sum += len(species_list)
species_end_index = cum_sum
talk_number_start_index = cum_sum

talk_number_dict = {talk_number_list[i]:i+cum_sum for i in range(len(talk_number_list))}
cum_sum += len(talk_number_list)
talk_end_index = cum_sum




In [4]:
#SVTRSNの順番に並べて出力のラベルとする
def calc_label(prompt):
    content = ProtocolParser.parse(prompt)

    labels = [0 for i in range(label_size)]
    
    #文型を分類してラベルを追加
    if type(content) == SVTRContent:
        labels[subject_dict[content.subject]] = 1
        labels[verb_dict[content.verb]] = 1
        labels[target_dict[content.target]] = 1
        labels[role_dict[content.role]] = 1
    elif type(content) == SVTContent:
        labels[subject_dict[content.subject]] = 1
        labels[verb_dict[content.verb]] = 1
        labels[target_dict[content.target]] = 1
    elif type(content) == SVTSContent:
        labels[subject_dict[content.subject]] = 1
        labels[verb_dict[content.verb]] = 1
        labels[target_dict[content.target]] = 1
        labels[species_dict[content.species]] = 1
    elif type(content) == ControlContent:
        labels[subject_dict[content.subject]] = 1
        labels[verb_dict[content.verb]] = 1
    else:
        #今の検証ではSyntaxエラーを投げる
        #TODO: 別構文も判定できるようにするためにはこの条件分岐を変える（追加する）必要あり
        print(prompt)
        raise SyntaxError("Syntax Error")
    
    return labels
        

In [5]:
# データの前処理(データローダに入力できる形に整形)

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# 文章をトークンに変換するトークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# ファインチューニングで学習するためミニバッチ生成用データローダの作成

# データを取得
dataset_plain = pickle.load(open('dataset.pkl', 'rb'))
dataset = []
# データを前処理
for data in dataset_plain:
    prompt = data[0]
    jp_script= data[1]

    #ラベルを生成
    labels = calc_label(prompt)
    sample = {'text':jp_script, 'labels':labels}
    #データセットに追加
    dataset.append(sample)
    
# データをトークンに変換
max_length = 128
dataset_for_loader = []

for data in dataset:
    text = data['text']
    labels = data['labels']
    encoding = tokenizer(
            text,
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
    encoding['labels'] = labels
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

# #以下実行検証用学習データ

# #以下実行検証用学習データ
# test_plain_dataset = [
# ['ESTIMATE','私はAgent[10]が騎士だと推測する'],
# ['COMINGOUT','Agent[01]はAgent[03]が狂人だとカミングアウトする'],
# ['Over','Over'],
# ['COMINGOUT','Agent[01]が占いだとカミングアウトする'],
# ['COMINGOUT','Agent[01]はAgent[01]が占い師だとカミングアウトする'],
# ['DIVINED','私は占った結果Agent[01]は白だった'],
# ['DIVINED','Agent[01]が占った結果Agent[02]は人狼だった'],
# ['GUARD','Agent[01]を護衛する'],
# ]
# for data in test_plain_dataset:
#     encoding = tokenizer(
#             data[1],
#             max_length=max_length, 
#             padding='max_length',
#             truncation=True
#         )
#     encoding['labels'] = verb_list.index(data[0])
#     encoding = { k: torch.tensor(v) for k, v in encoding.items() }
#     dataset_for_loader.append(encoding)

print(dataset_for_loader[0])

{'input_ids': tensor([    2, 16204,  1865,  4314,  7535,  4118,    81,     7,     9, 16204,
         1865,  4314,   213,  4118,    14,  1076,    75,    13,  5705,    34,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [6]:
# データセットの分割
random.shuffle(dataset_for_loader) 
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

#　データセットからデータローダを作成
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

## データの学習


In [7]:
# fine-tuning用のクラスを定義
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters() 
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        ) 

    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)
        scores = output.logits
        labels_predicted = ( scores > 0 ).int()
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct/scores.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

# 学習
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

trainer = pl.Trainer(
    gpus=1, 
    max_epochs=100,
    callbacks = [checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, 
    num_labels=label_size, 
    lr=1e-5
)
trainer.fit(model, dataloader_train, dataloader_val)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

Sanity Checking: 0it [00:00, ?it/s]

/home/takuya/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/takuya/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/takuya/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (27) is smaller 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## 検証

In [8]:
# 検証1: テストデータで評価
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/home/takuya/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /home/takuya/HDD1/work/AIWolfK2B/model/epoch=99-step=2700.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/takuya/HDD1/work/AIWolfK2B/model/epoch=99-step=2700.ckpt
/home/takuya/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of c

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.9928571581840515
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy: 0.99


In [28]:
# 得られたスコアからプロンプトを生成
def calc_prompt_from_scores(scores):
    #scoresのラベルはSVTRSNの順番に並べられている
    prompt = []
    subject_index = scores[subject_start_index:subject_end_index].argmax().item()
    prompt.append(subject_list[subject_index])
    
    
    verb_index = scores[verb_start_index:verb_end_index].argmax().item()
    prompt.append(verb_list[verb_index])
    #動詞で分類
    SVTR_verbs = ['ESTIMATE', 'COMINGOUT']
    SVTS_verbs = ['DIVINED', 'IDENTIFIED']
    SVT_verbs = ['DIVINATION', 'GUARD', 'VOTE','ATTACK', 'GUARDED', 'VOTED','ATTACKED']
    SV_verbs = ['Skip', 'Over']  
    verb = verb_list[verb_index]
    
    if verb in SVTR_verbs:
        target_index = scores[target_start_index:target_end_index].argmax().item()
        prompt.append(target_list[target_index])
        role_index = scores[role_start_index:role_end_index].argmax().item()
        prompt.append(role_list[role_index])
    elif verb in SVTS_verbs:
        target_index = scores[target_start_index:target_end_index].argmax().item()
        prompt.append(target_list[target_index])
        species_index = scores[species_start_index:species_end_index].argmax().item()
        prompt.append(species_list[species_index])
    elif verb in SVT_verbs:
        target_index = scores[target_start_index:target_end_index].argmax().item()
        prompt.append(target_list[target_index])
    elif verb in SV_verbs:
        pass
    else:
        #今の検証ではSyntaxエラーを投げる
        #TODO: 別構文も判定できるようにするためにはこの条件分岐を変える（追加する）必要あり
        print(verb)
        raise SyntaxError("Syntax Error")
    
    prompt = ' '.join(prompt)
    
    return prompt

In [34]:
# 検証2: 適当な入力分を用意してprompt予測をし結果を確認
# 入力する文章
text_list = [
    "Agent[03]はAgent[08]が狼だと推測する",
    "Agent[06]はAgent[06]が占い師だとカミングアウトする",
    "Agent[12]が占った結果Agent[10]は人狼だった",
    "Agent[12]が占った結果Agent[10]は人間だった",
    'Agent[08]が襲われたAgent[05]を霊媒すると人間だった',
    'Agent[05]はAgent[10]を護衛した',
    "Agent[10]はAgent[12]に投票する",
    
    "Agent[06]はAgent[08]が狼だと思う",
    "私が占い師です",
    "Agent[12]が占った結果、Agent[10]は人狼でした",
    "Agent[12]が占った結果、Agent[10]は人間でした",
    'Agent[12]がAgent[05]を霊媒すると人間でした',
    'Agent[12]はAgent[10]を守った',
    "Agent[10]はAgent[12]に投票します",
    
    "Agent[08]が狼だと思う",
    "私が占い師です",
    "占った結果、Agent[10]は人狼でした",
    "占った結果、Agent[10]は人間でした",
    'Agent[05]を霊媒すると人間でした',
    '私はAgent[10]を守った',
    "私はAgent[12]に投票します",
    
]

# モデルのロード
best_model_path = checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml.cuda()

# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() }

# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, score in zip(text_list, scores.double().cpu().numpy()):
    print('--')
    print(f'入力：{text}')
    print(f'予測：{calc_prompt_from_scores(score)}')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--
入力：Agent[03]はAgent[08]が狼だと推測する
予測：Agent[03] ESTIMATE Agent[08] WEREWOLF
--
入力：Agent[06]はAgent[06]が占い師だとカミングアウトする
予測：Agent[06] COMINGOUT Agent[06] SEER
--
入力：Agent[12]が占った結果Agent[10]は人狼だった
予測：Agent[12] DIVINED Agent[10] WEREWOLF
--
入力：Agent[12]が占った結果Agent[10]は人間だった
予測：Agent[12] DIVINED Agent[10] HUMAN
--
入力：Agent[08]が襲われたAgent[05]を霊媒すると人間だった
予測：Agent[08] IDENTIFIED Agent[05] HUMAN
--
入力：Agent[05]はAgent[10]を護衛した
予測：Agent[05] GUARDED Agent[10]
--
入力：Agent[10]はAgent[12]に投票する
予測：Agent[10] VOTE Agent[12]
--
入力：Agent[06]はAgent[08]が狼だと思う
予測：Agent[06] ESTIMATE Agent[08] WEREWOLF
--
入力：私が占い師です
予測：Agent[10] COMINGOUT Agent[05] SEER
--
入力：Agent[12]が占った結果、Agent[10]は人狼でした
予測：Agent[12] DIVINED Agent[10] WEREWOLF
--
入力：Agent[12]が占った結果、Agent[10]は人間でした
予測：Agent[12] DIVINED Agent[10] HUMAN
--
入力：Agent[12]がAgent[05]を霊媒すると人間でした
予測：Agent[12] IDENTIFIED Agent[05] HUMAN
--
入力：Agent[12]はAgent[10]を守った
予測：Agent[12] VOTE Agent[10]
--
入力：Agent[10]はAgent[12]に投票します
予測：Agent[10] VOTE Agent[12]
--
入力：Agent[08]が狼だと思

In [39]:
# Transformers対応のモデルを./jp2prompt_modelに保存
out_model_dir = './jp2prompt_model'

import datetime
today = datetime.date.today()
out_filename = format(today, '%Y%m%d')

torch.save(model.state_dict(),out_model_dir + '/' + f'{out_filename}.pth')
#model.bert_scml.save_pretrained('./model_transformers') 

In [43]:
# モデルのロード
best_model_path = "./jp2prompt_model/20221225.pth"
load_model = BertForSequenceClassificationMultiLabel_pl(MODEL_NAME, num_labels=label_size, lr=1e-5)
load_model.load_state_dict(torch.load(best_model_path))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>